In [1]:
import sys
sys.path.append("..")
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
import pickle
import random
import numpy as np
import copy
import json
from tqdm import tqdm


In [2]:
from das.Helper_Functions import set_seed
from das.Dataset_Generation import Generate_LLM_Eval_Intervention_Data
from das.LLM_Model import (make_model,
                       LLM_Criterion)
from das.RevNet import RevNet
from das.Rotation_Model import Rotation
from das.DAS import phi_class
from das.DAS_LLM import Distributed_Alignment_Search_LLM
from transformers import AutoTokenizer

In [3]:

model_config = {"model"   : "meta-llama/Llama-3.2-1B",
                "Trained" :                         0} 
#Trained=0:pretrained, 1:fully randomized, 2:only randomize llm head, 3: only randomize embedding, 4: randomize linked embedding and lm head

DEVICE        = "cuda:0" #"cuda:0" #"cuda"/"cpu"

"""
transformation_config = {"type"        : "Rotation",
                         "in_features" :       2048}
"""
transformation_config = {"type"          : "RevNet",
                         "number_blocks" :       10,
                         "in_features"   :     2048,
                         "hidden_size"   :       16}

Max_Epochs                       = 100 #4 #1 #50
Early_Stopping_Epochs            = 100 #4 #1 #50
early_stopping_improve_threshold = 0.001
LLM_test_samples                 = 1600
Intervention_train_size          = 1280000//8
Intervention_eval_size           = 1600
Intervention_test_size           = 1600
learning_rate                    = 0.0001#0.000001
ReduceLROnPlateau_patience       = 10

In [5]:
from huggingface_hub import hf_hub_download
tokenizer = AutoTokenizer.from_pretrained(model_config["model"])
tokenizer.pad_token = tokenizer.eos_token

repo_id = "fahamu/ioi"
filename = "mecha_ioi_200k.parquet"

print(f"Loading '{repo_id}' file '{filename}'...")
dataset_path = hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    repo_type="dataset"
)
LLM_test_data,DAS_Train,DAS_Eval,DAS_Test=Generate_LLM_Eval_Intervention_Data(filename=dataset_path,
                                                                              tokenizer=tokenizer,
                                                                              LLM_test_samples=LLM_test_samples,
                                                                              Intervention_train_size=Intervention_train_size,
                                                                              Intervention_eval_size=Intervention_eval_size,
                                                                              Intervention_test_size=Intervention_test_size)


Loading 'fahamu/ioi' file 'mecha_ioi_200k.parquet'...


100%|██████████| 1600/1600 [00:00<00:00, 3273.74it/s]


In [10]:
results=[]
for acseed in [4287]:
    results.append({})
    set_seed(acseed)
    model,accuracy=make_model(model_config["model"],LLM_test_data,model_config["Trained"],device=DEVICE)

100%|██████████| 1600/1600 [00:23<00:00, 67.31it/s]

Test Accuracy: 0.9994


In [12]:
Layers=[]
#Layers.append(("Layer7",model.model.layers[7]))
Layers.append(("Layer9",model.model.layers[9]))
#Layers.append(("Layer15",model.model.layers[15]))
inter_dims=[]
inter_dims.append([list(range(0,transformation_config["in_features"]//2))])
#inter_dims.append([list(range(0,transformation_config["in_features"]//64))])
#inter_dims.append([list(range(0,1))])

results[-1]["accuracy"]=accuracy
for LayerName,Layer in Layers:
    results[-1][LayerName]={}
    for inter_dim in inter_dims:
        print(LayerName,":",len(inter_dim[0]), flush=True)
        #Initialize transformation function
        
        #Initialize transformation function
        if transformation_config["type"]=="Rotation":
            p = Rotation(transformation_config["in_features"])
        elif transformation_config["type"]=="RevNet":
            p = RevNet(number_blocks =  transformation_config["number_blocks"],
                        in_features   =  transformation_config["in_features"],
                        hidden_size   =  transformation_config["hidden_size"]
                        )
        else:
            Exception("Unknown transformation function")
        p.to(DEVICE)
        p_inverse = p.inverse
        optimizer = optim.Adam(p.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=ReduceLROnPlateau_patience)
        criterion = LLM_Criterion
        
        
        phi=phi_class(p,p_inverse,LLM_Criterion,optimizer,scheduler)

        

        DAS_Experiment=Distributed_Alignment_Search_LLM(Model=model,
                                                        Model_Layer=Layer,
                                                        Train_Data_Raw=DAS_Train,
                                                        Test_Data_Raw=DAS_Test,
                                                        Eval_Data_Raw=DAS_Eval,
                                                        Hidden_Layer_Size=transformation_config["in_features"],
                                                        Variable_Dimensions=inter_dim,
                                                        Transformation_Class=phi,
                                                        Device=DEVICE,
                                                        tokenizer=tokenizer)

        DAS_Experiment.train_test(batch_size=64,
                                    epochs=Max_Epochs,
                                    mode=1,
                                    early_stopping_threshold=Early_Stopping_Epochs,
                                    early_stopping_improve_threshold=early_stopping_improve_threshold,
                                    verbose=True) #Train

        accuracy=DAS_Experiment.train_test(batch_size=32,
                                            mode=2,
                                            verbose=True)#Test
        results[-1][LayerName][str(inter_dim)]=accuracy
        DAS_Experiment.Cleanup()
        DAS_Experiment=None
        with open('results.json', 'w') as f:
            json.dump(results, f)

Layer9 : 1024
Training:


Loss: 1.5474, Accuracy: 0.0000:   0%|          | 3/2500 [05:16<49:13:17, 70.96s/it]  ../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [72,0,0], thread: [32,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [72,0,0], thread: [33,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [72,0,0], thread: [34,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [72,0,0], thread: [35,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [72,0,0], thread: [36,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/c

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
